In [1]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/FastaiNotebook_00_load_data")' FastaiNotebook_00_load_data

Installing packages:
	.package(path: "/home/jmd/workspace/ml/fastai/nbs/swift/FastaiNotebook_00_load_data")
		FastaiNotebook_00_load_data
With SwiftPM flags: []
Working in: /tmp/tmp1z_ykud4/swift-install
/home/jmd/swift/usr/bin/swift-build: /home/jmd/anaconda3/lib/libcurl.so.4: no version information available (required by /home/jmd/swift/usr/lib/swift/linux/libFoundationNetworking.so)
[1/2] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[2/3] Merging module jupyterInstalledPackages
Initializing Swift...
Installation complete!


In [2]:
//export
import Path
import TensorFlow

In [3]:
import FastaiNotebook_00_load_data

In [4]:
let zeros = Tensor<Float>(zeros: [1,4,5])
let ones  = Tensor<Float>(ones: [12,4,5])
let twos  = Tensor<Float>(repeating: 2.0, shape: [2,3,4,5])
let range = Tensor<Int32>(rangeFrom: 0, to: 32, stride: 1)

In [5]:
let xTrain = Tensor<Float>(randomNormal: [5, 784])
var weights = Tensor<Float>(randomNormal: [784, 10]) / sqrt(784)
print(weights[0])

[ 0.060334317,  0.027727272, -0.044780068, -0.015127443, -0.021600503, -0.032880202,
  -0.08513952, 0.0022793747,  -0.07226193,  -0.03517933]


In [6]:
// a and b are the flattened array elements, aDims/bDims are the #rows/columns of the arrays.
func swiftMatmul(a: [Float], b: [Float], aDims: (Int,Int), bDims: (Int,Int)) -> [Float] {
    assert(aDims.1 == bDims.0, "matmul shape mismatch")
    
    var res = Array(repeating: Float(0.0), count: aDims.0 * bDims.1)
    for i in 0 ..< aDims.0 {
        for j in 0 ..< bDims.1 {
            for k in 0 ..< aDims.1 {
                res[i*bDims.1+j] += a[i*aDims.1+k] * b[k*bDims.1+j]
            }
        }
    }
    return res
}

In [7]:
let flatA = xTrain[0..<5].scalars
let flatB = weights.scalars
let (aDims,bDims) = ((5, 784), (784, 10))

In [8]:
var resultArray = swiftMatmul(a: flatA, b: flatB, aDims: aDims, bDims: bDims)

In [9]:
time(repeating: 100) {
    _ = swiftMatmul(a: flatA, b: flatB, aDims: aDims, bDims: bDims)
}

average: 0.0988513 ms,   min: 0.098569 ms,   max: 0.113266 ms


In [10]:
// a and b are the flattened array elements, aDims/bDims are the #rows/columns of the arrays.
func swiftMatmulUnsafe(a: UnsafePointer<Float>, b: UnsafePointer<Float>, aDims: (Int,Int), bDims: (Int,Int)) -> [Float] {
    assert(aDims.1 == bDims.0, "matmul shape mismatch")
    
    var res = Array(repeating: Float(0.0), count: aDims.0 * bDims.1)
    res.withUnsafeMutableBufferPointer { res in 
        for i in 0 ..< aDims.0 {
            for j in 0 ..< bDims.1 {
                for k in 0 ..< aDims.1 {
                    res[i*bDims.1+j] += a[i*aDims.1+k] * b[k*bDims.1+j]
                }
            }
        }
    }
    return res
}

In [11]:
time(repeating: 100) {
    _ = swiftMatmulUnsafe(a: flatA, b: flatB, aDims: aDims, bDims: bDims)
}

average: 0.08404072999999997 ms,   min: 0.083808 ms,   max: 0.091723 ms


In [12]:
import Glibc

let ptr : UnsafeMutableRawPointer = malloc(42)

print("☠️☠️ Uninitialized garbage =", ptr.load(as: UInt8.self))

free(ptr)

☠️☠️ Uninitialized garbage = 32


In [13]:
#if canImport(PythonKit)
    import PythonKit
#else
    import Python
#endif
let np = Python.import("numpy")
let pickle = Python.import("pickle")
let sys = Python.import("sys")

print("🐍list =    ", pickle.dumps([1, 2, 3]))
print("🐍ndarray = ", pickle.dumps(np.array([[1, 2], [3, 4]])))


🐍list =     b'\x80\x03]q\x00(K\x01K\x02K\x03e.'
🐍ndarray =  b'\x80\x03cnumpy.core.multiarray\n_reconstruct\nq\x00cnumpy\nndarray\nq\x01K\x00\x85q\x02C\x01bq\x03\x87q\x04Rq\x05(K\x01K\x02K\x02\x86q\x06cnumpy\ndtype\nq\x07X\x02\x00\x00\x00i8q\x08K\x00K\x01\x87q\tRq\n(K\x03X\x01\x00\x00\x00<q\x0bNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x0cb\x89C \x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00q\rtq\x0eb.'


In [14]:
var bias = Tensor<Float>(zeros: [10])

let m1 = Tensor<Float>(randomNormal: [5, 784])
let m2 = Tensor<Float>(randomNormal: [784, 10])

In [15]:
print("m1: ", m1.shape)
print("m2: ", m2.shape)

m1:  [5, 784]
m2:  [784, 10]


In [16]:
let small = Tensor<Float>([[1, 2],
                           [3, 4]])

print("🔢2x2:\n", small)

🔢2x2:
 [[1.0, 2.0],
 [3.0, 4.0]]


In [17]:
print("⊞ matmul:\n",  matmul(small, small))
print("\n⊞ again:\n", small • small)


⊞ matmul:
 [[ 7.0, 10.0],
 [15.0, 22.0]]

⊞ again:
 [[ 7.0, 10.0],
 [15.0, 22.0]]


In [18]:
var bias = Tensor<Float>(zeros: [10])

let m1 = Tensor<Float>(randomNormal: [5, 784])
let m2 = Tensor<Float>(randomNormal: [784, 10])

In [19]:
print("m1: ", m1.shape)
print("m2: ", m2.shape)

m1:  [5, 784]
m2:  [784, 10]


In [20]:
let small = Tensor<Float>([[1, 2],
                           [3, 4]])

print("🔢2x2:\n", small)

🔢2x2:
 [[1.0, 2.0],
 [3.0, 4.0]]


In [21]:
print("⊞ matmul:\n",  matmul(small, small))
print("\n⊞ again:\n", small • small)


⊞ matmul:
 [[ 7.0, 10.0],
 [15.0, 22.0]]

⊞ again:
 [[ 7.0, 10.0],
 [15.0, 22.0]]


In [22]:
var m = Tensor([1.0, 2, 3, 4, 5, 6, 7, 8, 9]).reshaped(to: [3, 3])
print(m)

[[1.0, 2.0, 3.0],
 [4.0, 5.0, 6.0],
 [7.0, 8.0, 9.0]]


In [23]:
sqrt((m * m).sum())

16.881943016134134


In [24]:
var a = Tensor([10.0, 6, -4])
var b = Tensor([2.0, 8, 7])
(a,b)

▿ 2 elements
  - .0 : [10.0,  6.0, -4.0]
  - .1 : [2.0, 8.0, 7.0]


In [25]:
print("add:  ", a + b)
print("mul:  ", a * b)
print("sqrt: ", sqrt(a))
print("pow:  ", pow(a, b))

add:   [12.0, 14.0,  3.0]
mul:   [ 20.0,  48.0, -28.0]
sqrt:  [3.1622776601683795,  2.449489742783178,               -nan]
pow:   [             100.0, 1679616.0000000002,           -16384.0]


In [26]:
a < b

: 

In [27]:
a .< b

[false,  true,  true]


In [28]:
print((a .> 0).all())

false


In [29]:
print ((a .> 0).any())

true


In [30]:
var a = Tensor([10.0, 6.0, -4.0])

In [31]:
print(a+1)

[11.0,  7.0, -3.0]


In [32]:
2 * m

[[ 2.0,  4.0,  6.0],
 [ 8.0, 10.0, 12.0],
 [14.0, 16.0, 18.0]]


In [33]:
let c = Tensor([10.0,20.0,30.0])

In [34]:
m + c

[[11.0, 22.0, 33.0],
 [14.0, 25.0, 36.0],
 [17.0, 28.0, 39.0]]


In [35]:
c + m

[[11.0, 22.0, 33.0],
 [14.0, 25.0, 36.0],
 [17.0, 28.0, 39.0]]


In [36]:
m + c.expandingShape(at: 1)

[[11.0, 12.0, 13.0],
 [24.0, 25.0, 26.0],
 [37.0, 38.0, 39.0]]


In [37]:
c.expandingShape(at: 1)

[[10.0],
 [20.0],
 [30.0]]


In [38]:
print(c.expandingShape(at: 0).shape)

[1, 3]


In [39]:
print(c.expandingShape(at: 1).shape)

[3, 1]


In [40]:
c.expandingShape(at: 0) * c.expandingShape(at: 1)

[[100.0, 200.0, 300.0],
 [200.0, 400.0, 600.0],
 [300.0, 600.0, 900.0]]


In [41]:
c.expandingShape(at: 0) .> c.expandingShape(at: 1)

[[false,  true,  true],
 [false, false,  true],
 [false, false, false]]


In [42]:
func tensorMatmul(_ a: Tensor<Float>, _ b: Tensor<Float>) -> Tensor<Float> {
    var res = Tensor<Float>(zeros: [a.shape[0], b.shape[1]])
    
    for i in 0 ..< a.shape[0] {
        for j in 0 ..< b.shape[1] {
            for k in 0 ..< a.shape[1] {
                res [i, j] += a[i, k] * b[k, j]
            }
        }
    }
    return res
}

_ = tensorMatmul(m1, m2)

In [43]:
time { 
    let tmp = tensorMatmul(m1, m2)
    
    // Copy a scalar back to the host to force a GPU sync.
    _ = tmp[0, 0].scalar
}

average: 5183.05426 ms,   min: 5183.05426 ms,   max: 5183.05426 ms


In [44]:
func broadcastMatmult( _ a:Tensor<Float>, _ b: Tensor<Float>) -> Tensor<Float>{
    var res = Tensor<Float>(zeros: [a.shape[0], b.shape[1]])
    for i in 0..<a.shape[0] {
        res[i] = (a[i].expandingShape(at: 1) * b).sum(squeezingAxes: 0)
    }
    return res
}

_ = broadcastMatmult(m1, m2)

In [45]:
time(repeating: 100) {
    let tmp = broadcastMatmult(m1, m2)

    // Copy a scalar back to the host to force a GPU sync.
    _ = tmp[0, 0].scalar
}

average: 1.1179959199999998 ms,   min: 0.627245 ms,   max: 3.53016 ms


In [46]:
time(repeating: 100) { _ = m1 • m2 }

average: 0.012058640000000011 ms,   min: 0.011775 ms,   max: 0.01801 ms


In [47]:
func timeMatmulTensor(size: Int) {
    var matrix = Tensor<Float>(randomNormal: [size, size])
    print("\n\(size)x\(size):\n ⏰", terminator: "")
    time(repeating: 10) {
        let matrix = matrix • matrix
        _ = matrix[0, 0].scalar
    }
}

timeMatmulTensor(size: 1)     // Tiny
timeMatmulTensor(size: 10)    // Bigger
timeMatmulTensor(size: 100)   // Even Bigger
timeMatmulTensor(size: 1000)  // Biggerest
timeMatmulTensor(size: 5000)  // Even Biggerest


1x1:
 ⏰average: 0.49233829999999995 ms,   min: 0.421676 ms,   max: 0.606449 ms

10x10:
 ⏰average: 0.4890567 ms,   min: 0.417372 ms,   max: 0.505052 ms

100x100:
 ⏰average: 0.48377849999999994 ms,   min: 0.462205 ms,   max: 0.517784 ms

1000x1000:
 ⏰average: 0.8396785000000001 ms,   min: 0.820621 ms,   max: 0.85277 ms

5000x5000:
 ⏰average: 51.662761599999996 ms,   min: 49.752333 ms,   max: 54.419542 ms


In [48]:
func timeMatmulSwift(size: Int, repetitions: Int = 10) {
    var matrix = Tensor<Float>(randomNormal: [size, size])
    let matrixFlatArray = matrix.scalars

    print("\n\(size)x\(size):\n  ⏰", terminator: "")
    time(repeating: repetitions) { 
       _ = swiftMatmulUnsafe(a: matrixFlatArray, b: matrixFlatArray, aDims: (size,size), bDims: (size,size))
    }
}

timeMatmulSwift(size: 1)     // Tiny
timeMatmulSwift(size: 10)    // Bigger
timeMatmulSwift(size: 100)   // Even Bigger
timeMatmulSwift(size: 1000, repetitions: 1)  // Biggerest

print("\n5000x5000: skipped, it takes tooo long!")


1x1:
  ⏰average: 0.0007086 ms,   min: 0.000646 ms,   max: 0.001073 ms

10x10:
  ⏰average: 0.0083697 ms,   min: 0.008319 ms,   max: 0.008624 ms

100x100:
  ⏰average: 5.036378299999999 ms,   min: 2.563036 ms,   max: 10.773016 ms

1000x1000:
  ⏰average: 2194.553441 ms,   min: 2194.553441 ms,   max: 2194.553441 ms

5000x5000: skipped, it takes tooo long!


In [49]:
withDevice(.cpu) {
    timeMatmulTensor(size: 1)     // Tiny
    timeMatmulTensor(size: 10)    // Bigger
    timeMatmulTensor(size: 100)   // Even Bigger
    timeMatmulTensor(size: 1000)  // Biggerest
    timeMatmulTensor(size: 5000)  // Even Biggerest
}


1x1:
 ⏰average: 0.07649829999999999 ms,   min: 0.071807 ms,   max: 0.095279 ms

10x10:
 ⏰average: 0.05025759999999999 ms,   min: 0.043825 ms,   max: 0.099855 ms

100x100:
 ⏰average: 0.43538869999999996 ms,   min: 0.343571 ms,   max: 0.51299 ms

1000x1000:
 ⏰average: 5.8129200999999995 ms,   min: 4.637673 ms,   max: 7.236262 ms

5000x5000:
 ⏰average: 578.8298939 ms,   min: 573.066459 ms,   max: 596.03362 ms


In [50]:
// Explore the contents of the Raw namespace by typing Raw.<tab>
print(_Raw.zerosLike(c))

// Raw.

[0.0, 0.0, 0.0]


In [51]:
//export
public extension StringTensor {
    // Read a file into a Tensor.
    init(readFile filename: String) {
        self.init(readFile: StringTensor(filename))
    }
    init(readFile filename: StringTensor) {
        self = _Raw.readFile(filename: filename)
    }

    // Decode a StringTensor holding a JPEG file into a Tensor<UInt8>.
    func decodeJpeg(channels: Int = 0) -> Tensor<UInt8> {
        return _Raw.decodeJpeg(contents: self, channels: Int64(channels), dctMethod: "") 
    }
}


In [52]:
import NotebookExport
let exporter = NotebookExport(Path.cwd/"01_matmul.ipynb")
print(exporter.export(usingPrefix: "FastaiNotebook_"))

success
